In [31]:
from dotenv import load_dotenv
import json, requests
from bson import ObjectId

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

In [26]:
offices = list(db.companies_usa_filt1.find({}, {'_id':1, 'offices.latitude':1, 'offices.longitude':1,}))

In [42]:
load_dotenv()
foursq_api_id = os.getenv('foursquare_id')
foursq_api_k = os.getenv('foursquare_key')

In [43]:
## Borra cuando se solucione

offices = list(db.companies_usa_filt1.find({'offices.nightlife_spots':{'$exists':False}}, {'_id':1, 'offices.latitude':1, 'offices.longitude':1,}))

In [63]:
for office in offices:
    # Get latitude, longitude and id of the document
    latitude = office['offices']['latitude']
    longitude = office['offices']['longitude']
    id = ObjectId(office['_id'])

    # Set the requesto to Foursquare API, with its parameters (keys, latitude&longitude, 10,000 radius, 'Nightlife Spot' as categoryId and version_date)
    url = 'https://api.foursquare.com/v2/venues/search'
    params = dict(
        client_id = f"{foursq_api_id}",
        client_secret = f"{foursq_api_k}",
        ll = f"{latitude},{longitude}",
        radius = '10000',
        categoryId = '4d4b7105d754a06376d81259',
        v = 20201115)
    
    # Do the request and get the data
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    # Organize the data as dict of dicts, where the keys will go from 0 to the number of results - 1 and the values will be each of the found venues
    data_format = {}
    index = 0
    for venue in data['response']['venues']:
        data_format[str(index)] = venue
        index += 1

    # Update the document with a new section, called 'nightlife_spots'
    db.companies_usa_filt1.update_one({'_id':id}, {'$set':{'offices.nightlife_spots': data_format}})

KeyError: 'venues'

In [47]:
offices_mod = list(db.companies_usa_filt1.find({'offices.nightlife_spots':{'$exists':True}}, {'_id':1, 'offices.nightlife_spots':1}))

In [59]:
amount = []
for off in offices_mod:
    amount.append(list(offices_mod[0]['offices']['nightlife_spots'].keys())[-1])

In [61]:
set(amount)

{'29'}